Recall this code from the last exercise:

In [6]:
import random
from datetime import datetime

class Table():
    def __init__(self):
        self._data = {}
        
    def get(self, key):
        if key not in self._data.keys():
            return None
        
        return self._data[key]
    
    def set(self, key, value):
        self._data[key] = value

In [3]:
class NoTransactionInProgressException(Exception):
    def __str__(self):
        return "There isn't a transaction in progress!"
    
class TableAlreadyExistsException(Exception):
    def __str__(self):
        return "A table by that name already exists!"

class Database():
    def __init__(self):
        self._tables = {}
        
    def create_table(self, name):
        if name in self._tables.keys():
            raise TableAlreadyExistsException()
        
        self._tables[name] = Table()
        setattr(self, name, self._tables[name])
                
    def begin(self):
        pass
    
    def commit(self):
        raise NoTransactionInProgressException()
       
    def rollback(self):
        raise NoTransactionInProgressException()

In [4]:
import sys
!{sys.executable} -m pip install colorama 

sys.path.insert(0, '..')
from test_framework_exercise.phoenix_test.matchers import FailedAssertion, Assertion, assert_that
from test_framework_exercise.phoenix_test.test import Test
sys.path.remove('..')

You should consider upgrading via the '/Users/chelseatroy/.pyenv/versions/3.9.0/bin/python3.9 -m pip install --upgrade pip' command.


In [5]:
class TransactionTest(Test):
    def test_begin_commit(self):
        db = Database()
        db.create_table("birds_on_youtube")
        
        db.begin()
        db.birds_on_youtube.set("Vinny", {"type": "Galah", "owner": "Marlene McCohen", "interests" : "music"})
        db.commit()
        
        assert_that(db.birds_on_youtube.get("Vinny")).equals({"type": "Galah", "owner": "Marlene McCohen", "interests" : "music"})
        
    def test_commit_without_transaction(self):
        db = Database()
        
        try:
            db.commit()
            assert_that("No Exception Thrown").equals("NoTransactionInProgressException")
        except NoTransactionInProgressException:
            return
        
        assert_that("Wrong Exception Thrown").equals("NoTransactionInProgressException")

TransactionTest().run()

    test_begin_commit:  There isn't a transaction in progress!
    test_commit_without_transaction passed!
    1 out of 2 tests passed.



(1, 2)

Why do both of these already pass?

## Challenge 2

In [8]:
class TransactionAndRollbackTest(Test):
    def test_rollback_with_transaction(self):
        db = Database()
        db.create_table("birds_on_youtube")
        
        db.begin()
        db.birds_on_youtube.set("Vinny", {"type": "Galah", "owner": "Marlene McCohen", "interests" : "music"})
        db.rollback()

        assert_that(db.birds_on_youtube.get("Vinny")).equals(None)

    def test_rollback_no_transaction(self):
        db = Database()
        
        try:
            db.rollback()
            assert_that("No Exception Thrown").equals("NoTransactionInProgressException")
        except NoTransactionInProgressException:
            return
        
        assert_that("Wrong Exception Thrown").equals("NoTransactionInProgressException")

TransactionAndRollbackTest().run()

    test_rollback_no_transaction passed!
    test_rollback_with_transaction:  There isn't a transaction in progress!
    1 out of 2 tests passed.



(1, 2)

In [9]:
class TransactionAndCommitTest(Test):
    def test_rollback_with_transaction(self):
        db = Database()

        db.begin()
        db.set("Golden", "Eagle")
        db.rollback()

        print(db.get("Golden"))
        assert_that(db.get("Golden")).equals("Eagle")

    def test_rollback_no_transaction(self):
        db = Database()
        
        try:
            db.rollback()
            assert_that("No Exception Thrown").equals("NoTransactionInProgressException")
        except NoTransactionInProgressException:
            return
        
        assert_that("Wrong Exception Thrown").equals("NoTransactionInProgressException")
